# Introductory applied machine learning (INFR10069) 

# Lab 3: Support Vector Machine (SVM) Classification and Evaluation

In this lab we initially re-examine the spam filtering problem from Lab 2. This time, we train a Logistic Regression model and a linear Support Vector Machine for the spam or non-spam classification task. In the second part of the lab we examine classification evaluation by using a K-nearest neighbour classifier.


All the datasets that you will need for this lab are located at the `./datasets` directory which is adjacent to this file.

In [2]:
# Import packages
from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
KNeighboursClassifier = KNeighborsClassifier
from sklearn.metrics import confusion_matrix, log_loss
%matplotlib inline

## 1. Spam filtering

### ========== Question 1.1 ==========
Load `spambase_binary.csv` into a pandas DataFrame structure called `spambase`. Display the number of instances and attributes and the first 5 samples. Remember that the attributes have been binarised. The instances have also been shuffled (i.e. their order has been randomised). 

In [4]:
# Your code goes here
spambase = pd.read_csv('datasets/spambase_binary.csv')
print('Number of instances: ',spambase.shape[0])
print('Number of attributes: ',spambase.shape[1])
print(spambase.head(5))

Number of instances:  4601
Number of attributes:  55
   word_freq_make_binarized  word_freq_address_binarized  \
0                         0                            1   
1                         0                            0   
2                         0                            0   
3                         0                            0   
4                         0                            0   

   word_freq_all_binarized  word_freq_3d_binarized  word_freq_our_binarized  \
0                        0                       0                        1   
1                        0                       0                        0   
2                        1                       0                        0   
3                        1                       0                        1   
4                        0                       0                        1   

   word_freq_over_binarized  word_freq_remove_binarized  \
0                         0                         

### ========== Question 1.2 ==========
We are going to use hold-out validation to evaluate our models below. Split the dataset into training and testing subsets; there is a [built-in sklearn function](http://scikit-learn.org/0.17/modules/generated/sklearn.cross_validation.train_test_split.html) to do this. Call the resulting matrices `X_train`, `X_test`, `y_train`, `y_test`. Use 90% of the data for training and the remaining 10% for testing. Make sure you don't include the target variable `is_spam` in the input features (`X_train` / `X_test`)!

If you want to be able to reproduce your results exactly, what argument must you remember to set?

In [6]:
# Your code goes here
X_train, X_test, y_train, y_test = train_test_split(spambase.drop('is_spam', 1), spambase['is_spam'], test_size=0.1, random_state = 0 )
X_train.shape

(4140, 54)

*Your answer goes here*

The pseudo random number 'random_state' must be set to a value and that must be remebered to replicate anytime the same set has to be reproduced.

### ========== Question 1.3 ==========
Train a [`LogisticRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier by using training data. Use the `lbfgs` solver and default settings for the other parameters. Report the classification accuracy on both the training and test sets. Does your classifier generalise well on unseen data?

In [8]:
# Your code goes here
log_regress = LogisticRegression(solver = 'lbfgs')
log_regress.fit(X_train, y_train)
score_log_regress = log_regress.score(X_test, y_test)
print('On Test set',score_log_regress)
print('On Training set',log_regress.score(X_train, y_train))

On Test set 0.9284164859
On Training set 0.935024154589


*Your answer goes here*

### ========== Question 1.4 ==========
Print the coefficients for class 1 for the attributes `word_freq_hp_binarized` and `char_freq_$_binarized`. Generally, we would expect the string `$` to appear in spam, and the string `hp` to appear in non-spam e-mails, as the data was collected from HP Labs. Do the regression coefficients make sense given that class 1 is spam? *Hint: Consider the sigmoid function and how it transforms values into a probability between 0 and 1. Since our attributes are boolean, a positive coefficient can only increase the total sum fed through the sigmoid and thus move the output of the sigmoid towards 1. What can happen if we have continuous, real-valued attributes?*

In [13]:
# Your code goes here
print(log_regress.coef_[0,spambase.columns.get_loc('word_freq_hp_binarized')])
print(log_regress.coef_[0,spambase.columns.get_loc('char_freq_$_binarized')])
# spambase.columns.get_loc('char_freq_$_binarized')


-2.63934720019
1.69864273325


*Your answer goes here*

### ========== Question 1.5 ==========
Train a [`LinearSVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) (i.e. Linear Support Vector classifier) by using default parameters. Report the classification accuracy on the training and test sets. 

In [14]:
# Your code goes here
linearsvc = LinearSVC()
linearsvc.fit(X_train, y_train)
score_linearsvc = linearsvc.score(X_test, y_test)
print('On Test set',score_linearsvc)
print('On Training set',linearsvc.score(X_train, y_train))

On Test set 0.919739696312
On Training set 0.934541062802


### ========== Question 1.6 ==========
What are the coefficients for the attributes `word_freq_hp_binarized` and `char_freq_`$`_binarized`? Compare these to the ones you found with Logistic Regression.

In [16]:
# Your code goes here
column_hp = spambase.columns.get_loc('word_freq_hp_binarized')
column_dollar = spambase.columns.get_loc('char_freq_$_binarized')
print(linearsvc.coef_[0, column_hp])
print(linearsvc.coef_[0, column_dollar])

-0.856771182446
0.569354608669


*Your answer goes here*

### ========== Question 1.7 ==========
How does a linear SVM relate to Logistic Regression? *Hint: Consider the classification boundary learnt in each model.*

*Your answer goes here*
Since every datapoint is highly correlated, logistic regression has higher coeffficient values as they all contribute but SVC just takes few of the data points and they might be totally diverse and the correlation is diluted as we can see that through the coef value of the models.

### ========== Question 1.8 ==========
By using the [`SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) class train two new support vector classifiers with Gaussian (`rbf`) and polynomial (`poly`) kernels. Again, report classification accuracies on training and test sets and compare with your results from Question 1.5.

In [17]:
# Your code goes here
svc_rbf = SVC(kernel = "rbf")
svc_rbf.fit(X_train, y_train)
print(svc_rbf.score(X_test,y_test))
svc_poly = SVC(kernel = "poly")
svc_poly.fit(X_train, y_train)
print(svc_poly.score(X_test,y_test))

0.930585683297
0.809110629067


*Your answer goes here*

# 2. Performance assessment
We will now look at a few ways of assessing the performance of a classifier. To do so we will introduce a new data set, the [Splice](https://archive.ics.uci.edu/ml/datasets/Molecular+Biology+%28Splice-junction+Gene+Sequences%29) data set. The classification task is to identify `intron` and `exon` boundaries on gene sequences. For more information, you can read the dataset description in the link. The class attribute can take on 3 values: `N`, `IE` and `EI`.

### ========== Question 2.1 ==========
Load the `splice_train.csv` and `splice_test.csv` into two separate dataframes. Display the shape and first 10 instances for each dataframe.

In [4]:
# Your code goes here
splice_train = pd.read_csv('datasets/splice_train.csv')
splice_test = pd.read_csv('datasets/splice_test.csv')
print(splice_train.shape)
print(splice_test.shape)
print(splice_train.head(10))
print(splice_train.head(10))

(2935, 61)
(255, 61)
  pos0 pos1 pos2 pos3 pos4 pos5 pos6 pos7 pos8 pos9  ...  pos51 pos52 pos53  \
0    T    G    A    T    G    C    C    T    G    C  ...      C     C     C   
1    G    C    C    C    A    T    A    T    T    C  ...      T     G     G   
2    G    G    C    T    G    C    C    G    G    A  ...      A     C     T   
3    C    T    G    C    T    G    C    T    G    G  ...      G     G     C   
4    T    C    C    C    C    G    A    G    C    C  ...      A     T     C   
5    A    T    A    C    C    T    G    C    C    C  ...      A     T     G   
6    T    T    C    T    C    C    A    T    T    T  ...      G     A     T   
7    A    A    A    G    A    T    G    A    T    A  ...      A     A     G   
8    C    C    A    A    T    C    C    C    A    G  ...      G     G     C   
9    G    C    C    G    T    G    G    T    T    T  ...      A     A     G   

  pos54 pos55 pos56 pos57 pos58 pos59 class  
0     C     C     T     G     A     G     N  
1     A     C    

### ========== Question 2.2 ========== 
Convert the categorical attributes into numeric ones by using the [`LabelEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) tool. Make sure to not transform the target variable (`class`).

In [8]:
# Your code goes here
le = LabelEncoder()
for i in splice_train.columns.values:
    if i is not 'class':
        le.fit(splice_train[i])
        splice_train[i] = le.transform(splice_train[i])
le = LabelEncoder()
for i in splice_test.columns.values:
    if i is not 'class':
        le.fit(splice_test[i])
        splice_test[i] = le.transform(splice_test[i])
print(splice_test.head(10))

   pos0  pos1  pos2  pos3  pos4  pos5  pos6  pos7  pos8  pos9  ...    pos51  \
0     1     1     1     3     1     1     1     0     1     3  ...        1   
1     1     0     1     3     2     0     2     3     3     2  ...        2   
2     1     0     2     0     1     3     2     2     2     3  ...        0   
3     0     2     3     2     0     3     3     2     0     1  ...        3   
4     2     3     0     2     0     1     0     1     1     3  ...        0   
5     1     3     3     2     3     3     0     1     0     2  ...        1   
6     1     2     3     1     0     0     3     1     0     0  ...        0   
7     2     3     1     1     2     3     2     1     1     3  ...        2   
8     0     3     0     1     1     3     2     3     0     2  ...        1   
9     2     2     3     2     2     2     1     1     0     0  ...        1   

   pos52  pos53  pos54  pos55  pos56  pos57  pos58  pos59  class  
0      1      1      0      2      3      2      1      0      

### ========== Question 2.3 ==========
Store the training and testing data into numpy arrays `X_train`, `y_train`, `X_test` and `y_test`. Display the shapes of the four arrays.

In [9]:
# Your code goes here
X_train = splice_train.drop('class', 1)
y_train = splice_train['class']
X_test = splice_test.drop('class', 1)
y_test = splice_test['class']
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2935, 60) (2935,) (255, 60) (255,)


### ========== Question 2.4 ==========
Familiarise yourself with [Nearest Neighbours Classification](http://scikit-learn.org/stable/modules/neighbors.html#classification). Use a [`KNeighborsClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)
by using a single neighbour. Report the classification accuracy on the training set.

In [ ]:
# Your code goes here

### ========== Question 2.5 ==========
Is the above result meaningful? Why is testing on the training data a particularly bad idea for a 1-nearest neighbour classifier? Do you expect the performance of the classifier on a test set to be as good?

*Your answer goes here*

### ========== Question 2.6 ==========
Now report the classification accuracy on the test set and check your expectations.

In [ ]:
# Your code goes here

### ========== Question 2.7 ==========
Plot a histogram of the target variable (i.e. `class`) in the test set. *Hint: matplotlib won't allow you to plot a histogram for categorical values. Instead, you can use Pandas bulit-in bar plot tool in conjunction with the [`value_counts`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html). 

In [ ]:
# Your code goes here

### ========== Question 2.8 ==========
What would be the accuracy of the classifier, if all points were labelled as `N`? 

**Pro Tip** - You should always use a ['Dummy Model'](http://scikit-learn.org/stable/modules/model_evaluation.html#dummy-estimators) (a ridiculously simple model) like this to compare with your 'real' models. It's very common for complex models to be outperformed by a simple model, such as predicting the most common class. When complex models are outperformed by 'Dummies', you should investigate why: often there was an issue with the code, the data, or the way the model works was misunderstood.

In [ ]:
# Your code goes here

### ========== Question 2.9 ==========
Now we want to explore the effect of the `k` parameter. To do this, train the classifier multiple times, each time setting the KNN option to a different value. Try `5`, `10`, `50`, `100`, `200`, `500`, `1000`, `1500` and `2000` and test the classifier on the test set. How does the k parameter effect the results? *Hint: Consider how well the classifier is generalising to previously unseen data, and how it compares to the dumb prediction accuracy.*

In [ ]:
# Your code goes here

*Your answer goes here*

### ========== Question 2.10 ==========
Plot the results (k-value on the x-axis and classification accuracy on the y-axis), making sure to mark the axes. Can you conclude anything from observing the plot?

In [ ]:
# Your code goes here

*Your answer goes here*

### ========== Question 2.11 ==========
Select best value for `k` from Questions 2.9 and 2.10 and plot the normalised confusion matrix on the test set. Then plot the confusion matrix for a 5-nearest neighbour classifier. What do you observe?

In [ ]:
# Your code goes here

*Your answer goes here*

### ========== Question 2.12 ==========
Read about the [logarithimic loss](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html) (or cross-entropy loss). It is often the error metric used when we are trying to optimise classification models.

This metric takes as input the true labels and the estimated probability distributions (bernouli or multinomial). It makes sense to use this metric when we are interested not only in the predicted labels, but also in the confidence with which these labels are predicted.

For instance, think of the situation where you have a single test point and two classifiers. Both classifiers predict the label correctly, however classifier A predicts tha the test point belongs to the class with probability 0.55, whereas classifier B predicts the correct class with probability 0.99. Classification accuracy would be the same for the two classifiers (why?) but the `log_loss` metric would indicate that classifier B should be favoured.

Produce a scatter plot similar to the one in Question 2.10 but this time show `log_loss` on your y axis. Which value for `k` would you pick if `log_loss` was the error metric? Comment on whether accuracy or logarithmic loss is better for selecting a model.

In [ ]:
# Your code goes here

*Your answer goes here*

### ========== Question 2.13 ==========

Could you use the `log_loss` metric to evaluate the performance of an SVM classifier? Explain.

*Your answer goes here*